# AstroBkgInterp Uncertainty: JWST/NIRCam Imaging Example


<a id="imports"></a>
### Import Packages

- `AstroBkgInterp` is our background estimation tool
- `numpy` for array processing and math
- `atropy.io` for accessing the data
- `astropy.time` for timing
- `matplotlib` for plotting 

In [ ]:
from AstroBkgInterp.AstroBkgInterp import AstroBkgInterp
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1 import ImageGrid

## Setup
### Set path to Data
For this notebook, we will use a fully calibrated, resampled 2D JWST/NIRCam image in the F277W filter (with a CLEAR pupil), from Program 1433 (PI: D. Coe), observation 10.

In [ ]:
data_url = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_MRS_bkg_notebook/jw01433-o010_t001_nircam_clear-f277w_i2d.fits" 

In [ ]:
hdu = fits.open(data_url)
data = hdu[1].data
err = hdu[2].data

stamp = data[165:235,350:420]
errstamp = err[165:235,350:420]

## Run Background Modelling

In [ ]:
bi = AstroBkgInterp()

bi.src_y = 34
bi.src_x = 33

bi.aper_rad = 5
bi.ann_width = 3

bi.k = 2
bi.bin_size = 5

bi.kernel = None

bi.bkg_mode = 'polynomial'
bi.combine_fits = False

bi.pool_size = 1
bi.cube_resolution = 'high'

diff, bkg, mask, sigma = bi.run(stamp, err=errstamp)

## Plot results 

In [ ]:
fig,ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)

ax[0,0].set_title(r'Data', fontsize=15)
ax[0,1].set_title('Difference', fontsize=15)
ax[1,0].set_title(r'Error ($\sigma$)', fontsize=15)
ax[1,1].set_title(r'$\sigma$ Final', fontsize=15)

im1 = ax[0,0].imshow(stamp, vmin=-0.1, vmax=0.5, origin='lower', cmap='BuPu')
im2 = ax[0,1].imshow(diff, vmin=-0.1, vmax=0.5, origin='lower', cmap='BuPu')
im3 = ax[1,0].imshow(errstamp, vmin=0, vmax=0.05, origin='lower', cmap='BuPu')
im4 = ax[1,1].imshow(sigma, vmin=0, vmax=0.05, origin='lower', cmap='BuPu')

divider = make_axes_locatable(ax[0, 0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im2, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im3, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im4, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig('uncertainty_prop')
plt.show()